# Querying the dynamical.org Static STAC Catalog

This notebook shows how to query the [dynamical.org STAC catalog](https://opensciencecomputing.github.io/dynamical-stac/stac/dynamical/catalog.json)
hosted on GitHub Pages.

The catalog is a **static** STAC catalog (plain JSON files), so queries are performed
client-side using `pystac` and `shapely`. For larger catalogs a STAC API server would
be needed for efficient server-side filtering.

## Install dependencies
```
pip install pystac shapely
```

In [1]:
import pystac
from shapely.geometry import box, shape

CATALOG_URL = (
    "https://opensciencecomputing.github.io/dynamical-stac"
    "/stac/dynamical/catalog.json"
)

catalog = pystac.Catalog.from_file(CATALOG_URL)
print(catalog.description)
print(f"\n{len(list(catalog.get_items()))} items in catalog")

Weather forecast and analysis datasets from dynamical.org, stored as Icechunk repositories on AWS S3. All items can be opened directly with xarray via xpystac.



7 items in catalog


## 1. List all items

In [2]:
for item in catalog.get_items():
    print(f"{item.id}")
    print(f"  title:      {item.common_metadata.title}")
    print(f"  bbox:       {item.bbox}")
    print(f"  start:      {item.common_metadata.start_datetime}")
    print(f"  end:        {item.common_metadata.end_datetime}")
    print()

ecmwf-ifs-ens-forecast-15-day-0-25-degree-v0-1-0
  title:      ECMWF IFS ENS - dynamical.org Icechunk Zarr
  bbox:       [-180.0, -90.0, 180.0, 90.0]
  start:      2024-04-01 00:00:00+00:00
  end:        2026-02-23 00:00:00+00:00

noaa-gefs-analysis-v0-1-2
  title:      NOAA GEFS - dynamical.org Icechunk Zarr
  bbox:       [-180.0, -90.0, 180.0, 90.0]
  start:      2000-01-01 00:00:00+00:00
  end:        2026-02-23 09:00:00+00:00

noaa-gefs-forecast-35-day-v0-2-0
  title:      NOAA GEFS - dynamical.org Icechunk Zarr
  bbox:       [-180.0, -90.0, 180.0, 90.0]
  start:      2020-10-01 00:00:00+00:00
  end:        2026-02-23 00:00:00+00:00

noaa-gfs-analysis-v0-1-0
  title:      NOAA GFS - dynamical.org Icechunk Zarr
  bbox:       [-180.0, -90.0, 180.0, 90.0]
  start:      2021-05-01 00:00:00+00:00
  end:        2026-02-23 08:00:00+00:00

noaa-gfs-forecast-v0-2-7
  title:      NOAA GFS - dynamical.org Icechunk Zarr
  bbox:       [-180.0, -90.0, 180.0, 90.0]
  start:      2021-05-01 00:00:

## 2. Spatial query — find datasets that cover a region

Use `shapely` to test geometry intersection against a bounding box of interest.
Here we query for datasets covering Italy.

In [3]:
# Define region of interest (lon_min, lat_min, lon_max, lat_max)
roi = box(6.6, 36.6, 18.5, 47.1)  # Italy

matches = [
    item for item in catalog.get_items()
    if shape(item.geometry).intersects(roi)
]

print(f"Datasets covering Italy ({len(matches)} of {len(list(catalog.get_items()))}):\n")
for item in matches:
    print(f"  {item.id}")

Datasets covering Italy (5 of 7):

  ecmwf-ifs-ens-forecast-15-day-0-25-degree-v0-1-0
  noaa-gefs-analysis-v0-1-2
  noaa-gefs-forecast-35-day-v0-2-0
  noaa-gfs-analysis-v0-1-0
  noaa-gfs-forecast-v0-2-7


## 3. Temporal query — find datasets covering a date range

In [4]:
from datetime import datetime, timezone

query_start = datetime(2024, 1, 1, tzinfo=timezone.utc)
query_end   = datetime(2025, 1, 1, tzinfo=timezone.utc)

def overlaps_time(item, start, end):
    item_start = item.common_metadata.start_datetime
    item_end   = item.common_metadata.end_datetime
    if item_start is None or item_end is None:
        return True  # no temporal info — include by default
    return item_start <= end and item_end >= start

matches = [
    item for item in catalog.get_items()
    if overlaps_time(item, query_start, query_end)
]

print(f"Datasets covering {query_start.date()} – {query_end.date()} ({len(matches)}):\n")
for item in matches:
    print(f"  {item.id}")
    print(f"    {item.common_metadata.start_datetime} → {item.common_metadata.end_datetime}")

Datasets covering 2024-01-01 – 2025-01-01 (7):

  ecmwf-ifs-ens-forecast-15-day-0-25-degree-v0-1-0
    2024-04-01 00:00:00+00:00 → 2026-02-23 00:00:00+00:00
  noaa-gefs-analysis-v0-1-2
    2000-01-01 00:00:00+00:00 → 2026-02-23 09:00:00+00:00
  noaa-gefs-forecast-35-day-v0-2-0
    2020-10-01 00:00:00+00:00 → 2026-02-23 00:00:00+00:00
  noaa-gfs-analysis-v0-1-0
    2021-05-01 00:00:00+00:00 → 2026-02-23 08:00:00+00:00
  noaa-gfs-forecast-v0-2-7
    2021-05-01 00:00:00+00:00 → 2026-02-23 06:00:00+00:00
  noaa-hrrr-analysis-v0-1-0
    2018-09-16 00:00:00+00:00 → 2026-02-23 11:00:00+00:00
  noaa-hrrr-forecast-48-hour-v0-1-0
    2018-07-13 12:00:00+00:00 → 2026-02-23 06:00:00+00:00


## 4. Combined spatial + temporal query

In [5]:
roi         = box(6.6, 36.6, 18.5, 47.1)   # Italy
query_start = datetime(2024, 1, 1, tzinfo=timezone.utc)
query_end   = datetime(2025, 1, 1, tzinfo=timezone.utc)

matches = [
    item for item in catalog.get_items()
    if shape(item.geometry).intersects(roi)
    and overlaps_time(item, query_start, query_end)
]

print(f"Datasets covering Italy and {query_start.date()} – {query_end.date()} ({len(matches)}):\n")
for item in matches:
    print(f"  {item.id}")

Datasets covering Italy and 2024-01-01 – 2025-01-01 (5):

  ecmwf-ifs-ens-forecast-15-day-0-25-degree-v0-1-0
  noaa-gefs-analysis-v0-1-2
  noaa-gefs-forecast-35-day-v0-2-0
  noaa-gfs-analysis-v0-1-0
  noaa-gfs-forecast-v0-2-7


## 5. Open a matching dataset with xarray

Pick the first match and open it directly via `xpystac`.

In [6]:
import xarray as xr
import xpystac  # registers xarray backend for icechunk assets

item = matches[0]
print(f"Opening: {item.id}")

asset_key = next(k for k in item.assets if "@" in k)
asset = item.assets[asset_key]

ds = xr.open_dataset(asset)
ds

Opening: ecmwf-ifs-ens-forecast-15-day-0-25-degree-v0-1-0


<xarray.Dataset> Size: 162TB
Dimensions:                                     (init_time: 694, lead_time: 85,
                                                 ensemble_member: 51,
                                                 latitude: 721, longitude: 1440)
Coordinates:
  * init_time                                   (init_time) datetime64[ns] 6kB ...
    ingested_forecast_length                    (init_time) timedelta64[ns] 6kB ...
    expected_forecast_length                    (init_time) timedelta64[ns] 6kB ...
  * lead_time                                   (lead_time) timedelta64[ns] 680B ...
    valid_time                                  (init_time, lead_time) datetime64[ns] 472kB ...
  * ensemble_member                             (ensemble_member) int16 102B ...
  * latitude                                    (latitude) float64 6kB 90.0 ....
  * longitude                                   (longitude) float64 12kB -180...
    spatial_ref                                 int64 8B ...
Data variables: (12/13)
    categorical_precipitation_type_surface      (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    downward_long_wave_radiation_flux_surface   (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    dew_point_temperature_2m                    (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    downward_short_wave_radiation_flux_surface  (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    geopotential_height_500hpa                  (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    pressure_surface                            (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    ...                                          ...
    temperature_2m                              (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    pressure_reduced_to_mean_sea_level          (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    wind_u_100m                                 (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    wind_u_10m                                  (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    wind_v_10m                                  (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    wind_v_100m                                 (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
Attributes:
    dataset_id:           ecmwf-ifs-ens-forecast-15-day-0-25-degree
    dataset_version:      0.1.0
    name:                 ECMWF IFS ENS Forecast, 15 day, 0.25 degree
    description:          Ensemble weather forecasts from the ECMWF Integrate...
    attribution:          ECMWF IFS ENS Forecast data processed by dynamical....
    spatial_domain:       Global
    spatial_resolution:   0.25 degrees (~20km)
    time_domain:          Forecasts initialized 2024-04-01 00:00:00 UTC to Pr...
    time_resolution:      Forecasts initialized every 24 hours
    forecast_domain:      Forecast lead time 0-360 hours (0-15 days) ahead
    forecast_resolution:  Forecast step 0-144 hours: 3 hourly, 144-360 hours:...